### IMPORT OPTIONS

In [10]:
#Standalone Script
from pathlib import Path
import ujson
from elasticsearch import Elasticsearch

In [11]:
def read_jsonl(file_path):
    with Path(file_path).open('r', encoding='utf8') as f:
        for line in f:
            try:  # hack to handle broken jsonl
                yield ujson.loads(line.strip())
            except ValueError:
                continue

In [20]:
#path = "/Users/balazs/Desktop/args-me-1.0-cleaned.json"
path = "/Users/balazs/Desktop/debateorg.json"
print ("Reading from {}".format(path))
d = read_jsonl(path)

lines = []
i = 0
while True:
    try:
        lines.append(next(d))
        i=i+1
        if i%1001 == 0:
            print("Lines Read: ", i)
            break
    except Exception as e:
        print(e)
        break
print ("data loaded")

Reading from /Users/balazs/Desktop/args-me-1.0-cleaned.jsonl
Line 1: arguments
Data loaded


In [26]:
#Elasticsearch
print ("Connecting to elastic search")
# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = "DX6i+8n=85jVn9jVL0Ze"

# Create the client instance
es = Elasticsearch(
    "https://localhost:9200",
    ca_certs="/Users/balazs/elasticsearch-8.6.2/config/certs/http_ca.crt",
    basic_auth=("elastic", ELASTIC_PASSWORD)
)
print ("done")

# Successful response!
es.info()

Connecting to elastic search
done


ObjectApiResponse({'name': 'Balazss-MacBook-Pro.local', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'LVjKO5l4RlGe8MMJe_Rklg', 'version': {'number': '8.6.2', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '2d58d0f136141f03239816a4e360a8d17b6d8f29', 'build_date': '2023-02-13T09:35:20.314882762Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [34]:
index_name = "debateorg-preprocessed"
es.indices.create(index=index_name, body={
    "mappings": {
        "dynamic": True
    }
}, ignore=[400])
print ("start indexing")
for i,row in enumerate(lines):
    if not i%10:
        print("Lines Indexed: ", i)
    response = es.index(
        index = index_name,
        id = row['id'],
        body = {"text" : row["text"]
               })
    
print("indexing complete")

/var/folders/pj/99_398257tg2j0b00mlg8hqm0000gn/T/ipykernel_81014/1380468311.py:3: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=index_name, body={
/var/folders/pj/99_398257tg2j0b00mlg8hqm0000gn/T/ipykernel_81014/1380468311.py:3: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index=index_name, body={


start indexing
Lines Indexed:  0


KeyError: 'id'

In [33]:
#Test
s = "should schools have open campus?"
res = es.search(index = index_name, body={"query": {"match": {"text": s} } }, size = 25)['hits']['hits']

print("Elasticsearch test results: ", res)

Elasticsearch test results:  []


/var/folders/pj/99_398257tg2j0b00mlg8hqm0000gn/T/ipykernel_81014/339413573.py:3: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index = index_name, body={"query": {"match": {"text": s} } }, size = 25)['hits']['hits']
